crf测试pku中文语料第4版

改进：直接将数据格式为[word, label]的语料库拿来训练，特征提取为前后邻一个字，关注前后的序列，而不是别的特征。

///////////////////////////////////////////////////////////////////////

随意抽取一个带人名/地名的句子当作测试集

/////////////////////////////////////////////////////////////////////////////

In [2]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
import pycrfsuite
from itertools import chain

In [4]:
if __name__=='__main__':
    train_sents = []
    test_sents = []
    pairList = []
    f = open('./train_utf16.ner', 'r', encoding='utf_16_le')
    for line in f :
        line = line.encode('utf-8').decode('utf-8-sig').strip()
        if line:
            l = line.split(' ')
            pairList.append([l[0], l[1]])
        else:
            train_sents.append(pairList)
            pairList = []
    f.close()   

In [5]:
print(train_sents[0])

[['藏', 'N'], ['书', 'N'], ['本', 'N'], ['来', 'N'], ['就', 'N'], ['是', 'N'], ['所', 'N'], ['有', 'N'], ['传', 'N'], ['统', 'N'], ['收', 'N'], ['藏', 'N'], ['门', 'N'], ['类', 'N'], ['中', 'N'], ['的', 'N'], ['第', 'N'], ['一', 'N'], ['大', 'N'], ['户', 'N'], ['，', 'N'], ['只', 'N'], ['是', 'N'], ['我', 'N'], ['们', 'N'], ['结', 'N'], ['束', 'N'], ['温', 'N'], ['饱', 'N'], ['的', 'N'], ['时', 'N'], ['间', 'N'], ['太', 'N'], ['短', 'N'], ['而', 'N'], ['已', 'N'], ['。', 'N']]


In [12]:
def word2features(sent, i):
    word = sent[i][0]
    features = [
        'word=' + word,
    ]
    if i > 0:
        word1 = sent[i-1][0]
        features.extend([
            '-1:word=' + word1
        ])
    else:
        features.append('BOS')
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.extend([
            '+1:word=' + word1
        ])
    else:
        features.append('EOS')
    return features

In [13]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [30]:
def sent2labels(sent):
    return [label for token, label in sent]

In [31]:
def sent2tokens(sent):
    return [token for token, label in sent]

In [32]:
print(sent2features(train_sents[0])[0])

['word=藏', 'BOS', '+1:word=书']


In [33]:
print(sent2features(train_sents[0])[1])

['word=书', '-1:word=藏', '+1:word=本']


In [34]:
print(sent2features(train_sents[0])[-1])

['word=。', '-1:word=已', 'EOS']


In [35]:
print(train_sents[889])

[['一', 'N'], ['周', 'N'], ['来', 'N'], ['，', 'N'], ['河', 'B-LOC'], ['南', 'I-LOC'], ['西', 'B-LOC'], ['峡', 'I-LOC'], ['符', 'B-PER'], ['祖', 'I-PER'], ['国', 'I-PER'], ['、', 'N'], ['江', 'B-LOC'], ['西', 'I-LOC'], ['广', 'B-LOC'], ['昌', 'I-LOC'], ['吴', 'B-PER'], ['小', 'I-PER'], ['毛', 'I-PER'], ['、', 'N'], ['河', 'B-LOC'], ['北', 'I-LOC'], ['石', 'B-LOC'], ['家', 'I-LOC'], ['庄', 'I-LOC'], ['马', 'B-PER'], ['强', 'I-PER'], ['等', 'N'], ['百', 'N'], ['余', 'N'], ['位', 'N'], ['朋', 'N'], ['友', 'N'], ['给', 'N'], ['本', 'N'], ['刊', 'N'], ['来', 'N'], ['信', 'N'], ['来', 'N'], ['稿', 'N'], ['。', 'N']]


In [39]:
test_sents = train_sents[889]
print(test_sents)

[['一', 'N'], ['周', 'N'], ['来', 'N'], ['，', 'N'], ['河', 'B-LOC'], ['南', 'I-LOC'], ['西', 'B-LOC'], ['峡', 'I-LOC'], ['符', 'B-PER'], ['祖', 'I-PER'], ['国', 'I-PER'], ['、', 'N'], ['江', 'B-LOC'], ['西', 'I-LOC'], ['广', 'B-LOC'], ['昌', 'I-LOC'], ['吴', 'B-PER'], ['小', 'I-PER'], ['毛', 'I-PER'], ['、', 'N'], ['河', 'B-LOC'], ['北', 'I-LOC'], ['石', 'B-LOC'], ['家', 'I-LOC'], ['庄', 'I-LOC'], ['马', 'B-PER'], ['强', 'I-PER'], ['等', 'N'], ['百', 'N'], ['余', 'N'], ['位', 'N'], ['朋', 'N'], ['友', 'N'], ['给', 'N'], ['本', 'N'], ['刊', 'N'], ['来', 'N'], ['信', 'N'], ['来', 'N'], ['稿', 'N'], ['。', 'N']]


In [43]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(test_sents)]
y_test = [sent2labels(test_sents)]

In [44]:
print(sent2tokens(test_sents))

['一', '周', '来', '，', '河', '南', '西', '峡', '符', '祖', '国', '、', '江', '西', '广', '昌', '吴', '小', '毛', '、', '河', '北', '石', '家', '庄', '马', '强', '等', '百', '余', '位', '朋', '友', '给', '本', '刊', '来', '信', '来', '稿', '。']


In [45]:
print(sent2labels(test_sents))

['N', 'N', 'N', 'N', 'B-LOC', 'I-LOC', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'I-PER', 'N', 'B-LOC', 'I-LOC', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'I-PER', 'N', 'B-LOC', 'I-LOC', 'B-LOC', 'I-LOC', 'I-LOC', 'B-PER', 'I-PER', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']


In [46]:
trainer = pycrfsuite.Trainer(verbose=False)

In [51]:
for xseq, yseq in zip(X_train, y_train): 
    trainer.append(xseq, yseq) 

In [53]:
trainer.set_params({
    'c1':1.0,
    'c2':1e-3,
    'max_iterations':50,
    'feature.possible_transitions':True
})

In [55]:
trainer.train('conll2002_4.crfsuite')

In [57]:
tagger = pycrfsuite.Tagger()
tagger.open('conll2002_4.crfsuite')

In [59]:
print('true:', sent2labels(test_sents))
print('predicted:', tagger.tag(sent2features(test_sents)))

true: ['N', 'N', 'N', 'N', 'B-LOC', 'I-LOC', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'I-PER', 'N', 'B-LOC', 'I-LOC', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'I-PER', 'N', 'B-LOC', 'I-LOC', 'B-LOC', 'I-LOC', 'I-LOC', 'B-PER', 'I-PER', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']
predicted: ['N', 'N', 'N', 'N', 'B-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'B-PER', 'I-PER', 'I-PER', 'N', 'B-LOC', 'I-LOC', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'I-PER', 'N', 'B-LOC', 'I-LOC', 'B-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']


In [60]:
def bio_classification_report(y_true, y_pred):
    '''
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    '''
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
    tagset = set(lb.classes_) - {'N'} 
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)} 

    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels=[class_indices[cls] for cls in tagset],
        target_names=tagset,
    )

In [61]:
y_pred = [tagger.tag(xseq) for xseq in X_test]

print(bio_classification_report(y_test, y_pred))

             precision    recall  f1-score   support

      B-LOC       1.00      0.83      0.91         6
      I-LOC       0.78      1.00      0.88         7
      B-PER       1.00      0.67      0.80         3
      I-PER       1.00      0.80      0.89         5

avg / total       0.93      0.86      0.88        21



//////////////////////////////////////////////////////////////

7/3划分句子，划分训练集和测试集

////////////////////////////////////////////////////////////////////

In [63]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
import pycrfsuite
from itertools import chain
import random

In [125]:
if __name__=='__main__':
    train_sents = []
    test_sents = []
    pairList = []
    train = open('./train_data', 'r', encoding='utf-8')
    test = open('./test_data', 'r', encoding='utf-8')
    for line in train :
        line = line.encode('utf-8').decode('utf-8-sig').strip()
        if line:
            l = line.split(' ')
            pairList.append([l[0], l[1]])
        else:
            train_sents.append(pairList)
            pairList = []
    for line in test :
        line = line.encode('utf-8').decode('utf-8-sig').strip()
        if line:
            l = line.split(' ')
            pairList.append([l[0], l[1]])
        else:
            test_sents.append(pairList)
            pairList = []          
    train.close()
    test.close()

In [126]:
print(train_sents[-1])

[['“', 'N'], ['能', 'N'], ['选', 'N'], ['出', 'N'], ['一', 'N'], ['个', 'N'], ['好', 'N'], ['村', 'N'], ['长', 'N'], ['来', 'N'], ['，', 'N'], ['是', 'N'], ['我', 'N'], ['们', 'N'], ['老', 'N'], ['百', 'N'], ['姓', 'N'], ['的', 'N'], ['福', 'N'], ['分', 'N'], ['！', 'N'], ['”', 'N'], ['记', 'N'], ['票', 'N'], ['板', 'N'], ['上', 'N'], ['出', 'N'], ['现', 'N'], ['了', 'N'], ['左', 'B-PER'], ['金', 'I-PER'], ['文', 'I-PER'], ['的', 'N'], ['名', 'N'], ['字', 'N'], ['，', 'N'], ['而', 'N'], ['且', 'N'], ['票', 'N'], ['数', 'N'], ['扶', 'N'], ['摇', 'N'], ['直', 'N'], ['上', 'N'], ['，', 'N'], ['远', 'N'], ['远', 'N'], ['超', 'N'], ['过', 'N'], ['另', 'N'], ['外', 'N'], ['两', 'N'], ['名', 'N'], ['候', 'N'], ['选', 'N'], ['人', 'N'], ['。', 'N']]


In [113]:
print(test_sents[0])

[['以', 'N'], ['家', 'N'], ['乡', 'N'], ['的', 'N'], ['历', 'N'], ['史', 'N'], ['文', 'N'], ['献', 'N'], ['、', 'N'], ['特', 'N'], ['定', 'N'], ['历', 'N'], ['史', 'N'], ['时', 'N'], ['期', 'N'], ['书', 'N'], ['刊', 'N'], ['、', 'N'], ['某', 'N'], ['一', 'N'], ['名', 'N'], ['家', 'N'], ['或', 'N'], ['名', 'N'], ['著', 'N'], ['的', 'N'], ['多', 'N'], ['种', 'N'], ['出', 'N'], ['版', 'N'], ['物', 'N'], ['为', 'N'], ['专', 'N'], ['题', 'N'], ['，', 'N'], ['注', 'N'], ['意', 'N'], ['精', 'N'], ['品', 'N'], ['、', 'N'], ['非', 'N'], ['卖', 'N'], ['品', 'N'], ['、', 'N'], ['纪', 'N'], ['念', 'N'], ['品', 'N'], ['，', 'N'], ['集', 'N'], ['成', 'N'], ['系', 'N'], ['列', 'N'], ['，', 'N'], ['那', 'N'], ['收', 'N'], ['藏', 'N'], ['的', 'N'], ['过', 'N'], ['程', 'N'], ['就', 'N'], ['已', 'N'], ['经', 'N'], ['够', 'N'], ['您', 'N'], ['玩', 'N'], ['味', 'N'], ['无', 'N'], ['穷', 'N'], ['了', 'N'], ['。', 'N']]


In [83]:
print(line_index)    # 1089050个字符

1089050


In [114]:
def word2features(sent, i):
    word = sent[i][0]
    features = [
        'word=' + word,
    ]
    if i > 0:
        word1 = sent[i-1][0]
        features.extend([
            '-1:word=' + word1
        ])
    else:
        features.append('BOS')
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.extend([
            '+1:word=' + word1
        ])
    else:
        features.append('EOS')
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [115]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

In [116]:
trainer = pycrfsuite.Trainer(verbose=False)

In [117]:
for xseq, yseq in zip(X_train, y_train): 
    trainer.append(xseq, yseq) 

In [118]:
trainer.set_params({
    'c1':1.0,
    'c2':1e-3,
    'max_iterations':50,
    'feature.possible_transitions':True
})

In [119]:
trainer.train('conll2002_5.crfsuite')

In [120]:
tagger = pycrfsuite.Tagger()
tagger.open('conll2002_5.crfsuite')

In [121]:
print('true:', sent2labels(test_sents[0]))
print('predicted:', tagger.tag(sent2features(test_sents[0])))

true: ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']
predicted: ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']


In [122]:
def bio_classification_report(y_true, y_pred):
    '''
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    '''
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
    tagset = set(lb.classes_) - {'N'} 
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)} 

    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels=[class_indices[cls] for cls in tagset],
        target_names=tagset,
    )

In [123]:
y_pred = [tagger.tag(xseq) for xseq in X_test]

print(bio_classification_report(y_test, y_pred))

             precision    recall  f1-score   support

      B-LOC       0.87      0.78      0.82      5391
      I-LOC       0.83      0.78      0.80      7214
      B-ORG       0.82      0.64      0.72      3082
      I-ORG       0.83      0.72      0.77     12638
      B-PER       0.91      0.75      0.82      2760
      I-PER       0.89      0.81      0.85      5463

avg / total       0.85      0.75      0.80     36548

